In [49]:
import spacy
import pandas as pd
from pprint import pprint
from collections import defaultdict, OrderedDict

# 样例数据

In [2]:
text1 = '''
The Tedlar® Fluoropolymer technology group within the Photovoltaic and Fluoro Materials (PVFM) business in Dupont is seeking an Application Development Scientist with experience in multilayer polymer sheet coextrusion and casting product development. Tedlar® Fluoropolymer Products is an exciting and growing business supplying polyvinylfluoride (PVF) polymer, films and coatings for innovative solutions in the Photovoltaic, Electronics, Transportation, Construction, and broad Industrial markets.

 

The Application Development Scientist will develop novel and cost-effective products and process technology for various market-driven initiatives. The candidate will lead and implement research projects, including design, investigation and analysis. We are seeking a creative, results-focused individual with project management experience, and a proven track record of technical innovation and product commercialization.

 

Job Description:

·       Lead and implement new product development projects including overall project planning, product design & development, data analysis, pilot trials, patent filling and product launch

·       Innovative method and formulation development to drive thoroughly understanding of co-extrusion and casting multilayer sheets

·       Collaborate with multiple functions including the EH&S, Marketing, Sales, Reliability and Product for new product launching

·       Develop formulation and design process technology for making multilayer co-extruded and casted films

·       Work with customer accounts, technology team and sales team to conduct new product promotion and launch

·       Provide guidance to process engineer to validate and optimize the new formulation and process

 

Job Qualifications:

·       PhD degree in Material Science, Polymer Science, Chemical Engineering or Chemistry Discipline

·       Expertise in formulation/product development and in making multilayer co-extruded and casted films

·       5 years plus experience in project management, six sigma and DOE is preferred

·       Good analytical thinking in relating key experimental findings to target CTQs

·       Ability to work under pressure and challenging schedule

·       Proactive and innovative

·       Strong interpersonal and communication skills. Fluent in English.

·       Willing to travel (20 – 30%) for technical interactions and customer support
'''

In [3]:
text2 = '''
LinkedIn was built to help professionals achieve more in their careers, and every day millions of people use our products to make connections, discover opportunities and gain insights. Our global reach means we get to make a direct impact on the world’s workforce in ways no other company can. We’re much more than a digital resume – we transform lives through innovative products and technology. 

 

LinkedIn’s Strategy & Insights Data Scientists leverage big data to empower business decisions and deliver insights, metrics, and tools in order to drive member engagement, business growth, and monetization efforts. With over 645 million members around the world, a focus on great user experience, and a mix of B2B and B2C programs, a career at LinkedIn offers countless ways for an ambitious data scientist to have an impact. 

 

We are looking for a talented and driven individual to accelerate our efforts and be a major part of our data-centric culture. This person will work as a thought partner with various cross-functional teams such as product, marketing, sales, engineering, policy and operations to develop and deliver metrics, analyses, solutions, and insights, with actionable recommendations to business partners. Successful candidates will exhibit technical acumen, product sense and business savvy, with a passion for making an impact through creative storytelling and timely actions. 

 

Responsibilities 

Communicate findings to senior leaders and evangelize data-driven business decisions. 
Work with a team of high-performing data science professionals, and cross-functional teams to identify business opportunities, optimize product performance or go to market strategy. 
Analyze large-scale structured and unstructured data; develop deep-dive analyses and machine learning models to drive member value and customer success. 
Design and develop core business metrics, create insightful automated dashboards and data visualization to track them and extract useful business and product insights 
Design and analyze experiments to test new product ideas or go to market strategies. Convert the results into actionable recommendations. 
Craft compelling stories; make logical recommendations; drive informed actions. 
Initiate and drive projects to completion with minimal guidance. 
 

Basic Qualifications 

Bachelor, Masters or PhD degree in a quantitative discipline: statistics, operations research, computer science, informatics, engineering, applied mathematics, economics, etc. or equivalent work experience. 
BS 3+ years of relevant work experience, MS 2+ years of relevant work experience, or PhD 2+ years of relevant work experience 
Experience influencing strategy through data-centric presentations. 
Proficiency in SQL. 
Experience telling stories with data and visualization tools 
 

Preferred Qualifications 

Experience in high-tech company working with massive-scale data. 
Proven record writing and optimizing code with high levels of craftsmanship 
Working knowledge of Unix command-line/shell, git and review board. 
Experience running platform experiments and techniques like A/B testing 
Excellent communication skills, with the ability to synthesize, simplify and explain complex problems to different types of audience, including executives and compile compelling narratives 
Background in other programming languages (e.g., R, Python, Scala) 
Experience in applied statistics and statistical modeling 
Entrepreneurial experience'''

In [4]:
text3 = '''
We are currently looking for a Solution Architect for Deep Learning.

NVIDIA is searching for a world-class engineer for an exciting role as a Solutions Architect. Work with the most exciting high-performance computing hardware, software and impactful projects. Help enable compelling customer productivity and develop lasting relationships with NVIDIA's Technology Partners by making NVIDIA technology an integral part of end-user solutions.

You will work with the latest High Performance Computing architectures coupled with Machine Learning/Deep Learning, and Big Data Analytics application providers and customers to create improved workflows and develop new, differentiated solutions. In addition you will evangelize and influence the NVIDIA GPU Computing experience.

What You’ll Be Doing
Assist field business development in guiding the customer through the sales process for GPU Computing products, owning the technical relationship and assisting customer in building innovative solutions based on NVIDIA technology.
Be an industry thought leader on integrating NVIDIA technology into Deep Learning Industry to support Scientific and engineering applications.
Be an internal champion for Deep Learning or Data Science among the NVIDIA technical community.

What We Need To See
BS in Engineering, Mathematics, Physics, or Computer Science.
Knowledge with: Computer Vision, NLP, Big Data, and Speech.
7+ years of working experiences.
3+ years of Deep Learning experience, have project experiences in one of the following: detection, recognition, tracking, translation, big data analysis, speech recognition, etc.
Experience working with modern Deep Learning software architecture and application.
Experience with Tensorflow, Caffe, Pytorch, or other Deep Learning Framework.
Programming experience with C, Python, C#, or other languages.
Exposure to GPU technology and CUDA programming.
Customer facing skill-set and background is a plus.
Parallel programming experience, OpenMP, MPI or other programming languages is a plus.
Ability to communicate effectively, interpersonal skills and strong teamwork.

Ways To Stand Out From The Crowd
Ability to multitask effectively in a fast paced environment.
Action oriented with strong analytical and problem solving skills.
NVIDIA is widely considered to be one of the technology world’s most desirable employers. We have some of the most forward-thinking and hardworking people in the world working for us. If you're creative and autonomous, we want to hear from you!
#deeplearning
'''

# 文本分块 ==> 找出 job Requirements 相关的文本块

#### 1.如果文本中存在 Requirements、Qualifications 之类的关键词, 则首先以其作为分块依据
#### 2.如果找不到该类关键词, 则判断文本块句子的长度，以短文本作为段落名，将句子归入该段落
#### 3.如果出现段落句子以 序号、标点符号起始, 则将其归为一个段落

In [39]:
# 清洗
from spacy.lemmatizer import Lemmatizer

sentence_list = text2.split('\n')
sentence_list = [s.strip() for s in sentence_list]
sentence_list

['',
 'LinkedIn was built to help professionals achieve more in their careers, and every day millions of people use our products to make connections, discover opportunities and gain insights. Our global reach means we get to make a direct impact on the world’s workforce in ways no other company can. We’re much more than a digital resume – we transform lives through innovative products and technology.',
 '',
 '',
 '',
 'LinkedIn’s Strategy & Insights Data Scientists leverage big data to empower business decisions and deliver insights, metrics, and tools in order to drive member engagement, business growth, and monetization efforts. With over 645 million members around the world, a focus on great user experience, and a mix of B2B and B2C programs, a career at LinkedIn offers countless ways for an ambitious data scientist to have an impact.',
 '',
 '',
 '',
 'We are looking for a talented and driven individual to accelerate our efforts and be a major part of our data-centric culture. This

###  1.如果文本中存在 Requirements、Qualifications 之类的关键词 或 以冒号结尾, 则首先以其作为分块依据¶

In [46]:
import re

text_chunking_dict = {}

paragraph_name_List = []
match_word_list = ['Qualifications','Responsibilities']

# 正则表达式匹配 关键词 或者 以 ':' 结尾
for index in range( len(sentence_list) ):
    if sentence_list[index] != '':
        if re.findall( r'|'.join(match_word_list), sentence_list[index] ) or sentence_list[index][-1] == ':' or sentence_list[index][-1] == '：':
            paragraph_name_List.append( index )
        
if len( paragraph_name_List ) != 0:
    temp_paragraph = 'Description'
    text_chunking_dict[temp_paragraph] = []
    
    for index in range( len(sentence_list) ):
        if index not in paragraph_name_List:
            if sentence_list[index] != '':
                text_chunking_dict[temp_paragraph].append( sentence_list[index] )
        else:
            temp_paragraph = sentence_list[index]
            text_chunking_dict[temp_paragraph] = []

### 2.如果无法匹配到该类关键词，则查找是否存在短文本、且其后存在大段文字，若满足则以其作为分块依据

In [48]:
if len( paragraph_name_List ) == 0:

    text_chunking_dict = OrderedDict()
    # text_chunking_dict = {}

    temp_paragraph = 'Description'
    text_chunking_dict[temp_paragraph] = []

    for row in text2.split('\n'):
        if len( row.strip() ) == 0:
            continue

        if len(row.strip().split()) < 4:
            temp_paragraph = row.strip()
            text_chunking_dict[temp_paragraph] = []
        else:
            text_chunking_dict[temp_paragraph].append( row.strip() )

### 打印分块结果

In [65]:
pprint(  dict(text_chunking_dict) )

{'Basic Qualifications': ['Bachelor, Masters or PhD degree in a quantitative '
                          'discipline: statistics, operations research, '
                          'computer science, informatics, engineering, applied '
                          'mathematics, economics, etc. or equivalent work '
                          'experience.',
                          'BS 3+ years of relevant work experience, MS 2+ '
                          'years of relevant work experience, or PhD 2+ years '
                          'of relevant work experience',
                          'Experience influencing strategy through '
                          'data-centric presentations.',
                          'Proficiency in SQL.',
                          'Experience telling stories with data and '
                          'visualization tools'],
 'Description': ['LinkedIn was built to help professionals achieve more in '
                 'their careers, and every day millions of peop

# 领域关键词识别

### 基于 开源工具 的关键词识别

In [207]:
import spacy

nlp = spacy.load("en_core_web_sm")

string = "Bachelor, Masters or PhD degree in a quantitative discipline: statistics, operations research, computer science, informatics, engineering, applied mathematics, economics, etc. or equivalent work experience. "
doc = nlp(string)

word_list = [chunk.text for chunk in doc.noun_chunks]

print( 'Raw text:')
print( string )
print()
print( 'Key words:' )
print( word_list )

Raw text:
Bachelor, Masters or PhD degree in a quantitative discipline: statistics, operations research, computer science, informatics, engineering, applied mathematics, economics, etc. or equivalent work experience. 

Key words:
['Bachelor', 'Masters', 'PhD degree', 'a quantitative discipline', 'statistics', 'operations research', 'computer science', 'informatics', 'engineering', 'mathematics', 'economics', 'or equivalent work experience']


In [55]:
import spacy

nlp = spacy.load("en_core_web_sm")

text_chunking_keyword_dict = {}

for key in text_chunking_dict.keys():
    text_chunking_keyword_dict[key] = []
    
    for row in text_chunking_dict[key]:
        text_chunking_keyword_dict[key].append( [w.text for w in nlp(row).noun_chunks] )
    
pprint( text_chunking_keyword_dict )

{'Basic Qualifications': [['Bachelor',
                           'Masters',
                           'PhD degree',
                           'a quantitative discipline',
                           'statistics',
                           'operations research',
                           'computer science',
                           'informatics',
                           'engineering',
                           'mathematics',
                           'economics',
                           'or equivalent work experience'],
                          ['BS',
                           'years',
                           'relevant work experience',
                           'relevant work experience',
                           'relevant work experience'],
                          ['Experience',
                           'strategy',
                           'data-centric presentations'],
                          ['Proficiency', 'SQL'],
                          ['Experience

### 基于 序列标注模型 的领域关键词识别

In [ ]:
import bilstm_crf_model

###
###
###

# 层次关系抽取

### 基于句法分析树的层次关系获取
### 1.根据上一步获得的 领域关键词，找出领域关键词之间的依存关系【平级关系、上下级关系】
### 2根据结果生成树状结构

In [89]:
import spacy
from spacy_conll import ConllFormatter

nlp = spacy.load("en_core_web_sm")
conllformatter = ConllFormatter(nlp)
nlp.add_pipe(conllformatter, after='parser')
doc = nlp('Strong knowledge in computer vision, including image registration, segmentation, classification, object detection.')
conll = doc._.conll
# print(doc._.conll_str_headers)
print(doc._.conll_str)

1	Strong	strong	ADJ	JJ	Degree=pos	2	amod	_	_
2	knowledge	knowledge	NOUN	NN	Number=sing	0	ROOT	_	_
3	in	in	ADP	IN	_	2	prep	_	_
4	computer	computer	NOUN	NN	Number=sing	5	compound	_	_
5	vision	vision	NOUN	NN	Number=sing	3	pobj	_	_
6	,	,	PUNCT	,	PunctType=comm	5	punct	_	_
7	including	include	VERB	VBG	VerbForm=part|Tense=pres|Aspect=prog	5	prep	_	_
8	image	image	NOUN	NN	Number=sing	9	compound	_	_
9	registration	registration	NOUN	NN	Number=sing	7	pobj	_	_
10	,	,	PUNCT	,	PunctType=comm	9	punct	_	_
11	segmentation	segmentation	NOUN	NN	Number=sing	9	conj	_	_
12	,	,	PUNCT	,	PunctType=comm	11	punct	_	_
13	classification	classification	NOUN	NN	Number=sing	11	conj	_	_
14	,	,	PUNCT	,	PunctType=comm	13	punct	_	_
15	object	object	NOUN	NN	Number=sing	16	compound	_	_
16	detection	detection	NOUN	NN	Number=sing	13	conj	_	_
17	.	.	PUNCT	.	PunctType=peri	2	punct	_	_



In [126]:
from stat_parser import Parser
parser = Parser()
print( parser.parse('Strong knowledge in computer vision, including image registration, segmentation, classification, object detection.') )

(SINV
  (S
    (NP
      (NP (JJ strong) (NN knowledge))
      (PP (IN in) (NP (NN computer) (NN vision))))
    (VP
      (, ,)
      (S+VP (VBG including) (NP (NN image) (NN registration)))))
  (, ,)
  (VP (VBZ segmentation))
  (, ,)
  (VP (VBZ classification))
  (, ,)
  (NP (JJ object) (NN detection))
  (. .))


# 输出可视化结果

In [123]:
from unittest.mock import patch
from nose.tools import assert_equal, assert_in
from pyecharts import options as opts
from pyecharts.charts import Tree

In [124]:
TEST_DATA = [
    {
        "children": [
            {"name": "B"},
            {
                "children": [{"children": [{"name": "I"}], "name": "E"}, {"name": "F"}],
                "name": "C",
            },
        ],
        "name": "A",
    }
]

In [125]:
c = Tree().add(
        series_name="tree",
        data=TEST_DATA,
#         orient="BT",
        initial_tree_depth=5,
        label_opts=opts.LabelOpts(),
        leaves_label_opts=opts.LabelOpts(),
    )
c.render_notebook()